In [1]:
import pandas as pd
import numpy as np

In [7]:
sub_lgb = pd.read_csv('submissions/benchmark_ft_selection_ft_eng_0_967_rounds.csv')
sub_xgb = pd.read_csv('submissions/benchmark_ft_selection_ft_eng_0_catboost.csv')
sub_cat = pd.read_csv('submissions/xgboost_benchmark_best_243ft.csv')
sub = pd.read_csv('input/sample_submission.csv')

In [2]:
sub_r = pd.read_csv('submissions/benchmark_ensemble_ft_selection_ft_eng_0_lgb_xgboost_catboost.csv')
sub_a = pd.read_csv('submissions/20190822_catboost_80_lightgbm_64_xgboost_80_ts.csv')
sub = pd.read_csv('input/sample_submission.csv')

In [3]:
sub.head()

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5
2,3663551,0.5
3,3663552,0.5
4,3663553,0.5


In [4]:
from sklearn.preprocessing import MinMaxScaler

In [5]:
scaler = MinMaxScaler()

In [11]:
sub_r_scaler = scaler.fit_transform(sub_r.isFraud.values.reshape(-1, 1))
sub_a_scaler = scaler.fit_transform(sub_a.isFraud.values.reshape(-1, 1))

In [12]:
sub_r_scaler[:5]

array([[0.00050657],
       [0.00331102],
       [0.00170982],
       [0.00302157],
       [0.00180151]])

In [13]:
sub_a_scaler[:5]

array([[0.00168898],
       [0.00141766],
       [0.00230334],
       [0.00265393],
       [0.00161914]])

In [14]:
preds = (sub_r_scaler+sub_a_scaler)/2

In [20]:
preds = sub_r.isFraud*0.4+sub_a.isFraud*0.6

In [22]:
preds = (sub_lgb.isFraud+sub_cat.isFraud+sub_xgb.isFraud)/3

In [16]:
# preds.head()

In [21]:
sub['isFraud'] = preds

In [22]:
sub.head(10)

,TransactionID,isFraud
0,3663549,0.001381
1,3663550,0.002337
2,3663551,0.002228
3,3663552,0.002962
4,3663553,0.001855
5,3663554,0.003734
6,3663555,0.012160
7,3663556,0.021180
8,3663557,0.000813
9,3663558,0.006604


In [23]:
sub.to_csv('submissions/benchmark_ensemble_r_a_04_06.csv', sep=',', header=True, index=None)